In [1]:
import pandas as pd
import os
import pickle as pkl
from pprint import pprint
from plotly import express as px
from lightgbm import LGBMRegressor

In [2]:
os.chdir("../../")

In [3]:
from lib.config import config
from lib.model.utils import qwk_obj

In [4]:
df = pd.DataFrame()
names = None

for fold in range(config.lgbm_n_folds):
    model: LGBMRegressor = pkl.load(open(f"output/LGBM/{fold}.pkl", "rb"))["model"]
    importances = model.feature_importances_

    if not names:
        names = model.feature_name_
        df = pd.concat([df, pd.DataFrame({"name": names})], axis=1)

    temp = pd.DataFrame({f"LGBM_{fold}": importances})
    df = pd.concat([df, temp], axis=1)

In [5]:
df.head(10)

,name,LGBM_0,LGBM_1,LGBM_2,LGBM_3,LGBM_4,LGBM_5,LGBM_6,LGBM_7,LGBM_8,...,LGBM_15,LGBM_16,LGBM_17,LGBM_18,LGBM_19,LGBM_20,LGBM_21,LGBM_22,LGBM_23,LGBM_24
0,deberta_m0_c0,112,153,107,148,127,200,162,170,168,...,68,74,78,121,100,133,156,138,102,154
1,deberta_m0_c1,111,122,104,121,115,150,142,123,125,...,64,81,100,110,106,116,126,113,111,148
2,deberta_m0_c2,91,117,101,104,99,126,115,115,102,...,57,85,79,101,102,98,130,107,94,122
3,deberta_m0_c3,63,77,74,75,71,111,83,95,83,...,48,46,54,62,63,89,91,76,63,93
4,deberta_m0_c4,74,70,74,70,72,124,87,94,74,...,50,53,60,69,56,84,96,69,66,86
5,deberta_m0_c5,22,35,32,30,41,62,41,40,44,...,14,22,26,25,23,28,41,29,27,41
6,deberta_m1_c0,62,71,60,62,77,111,85,78,70,...,30,38,36,60,53,76,73,55,54,78
7,deberta_m1_c1,42,59,56,60,54,76,64,84,61,...,43,36,55,63,47,65,58,43,66,67
8,deberta_m1_c2,96,111,102,101,101,126,127,126,118,...,92,79,117,109,96,126,137,116,112,133
9,deberta_m1_c3,37,53,41,65,53,80,66,63,55,...,35,37,44,46,42,53,68,52,60,68


In [6]:
df["importance"] = df.loc[:, "LGBM_0":].median(axis=1)
df.sort_values(by="importance", ascending=False, inplace=True)
df = df[["name", "importance"]]

## Overall

Top 10 most important features

In [7]:
df.head(10)

,name,importance
0,deberta_m0_c0,127.0
20,deberta_m3_c2,125.0
1,deberta_m0_c1,112.0
8,deberta_m1_c2,111.0
18,deberta_m3_c0,106.0
12,deberta_m2_c0,103.0
2,deberta_m0_c2,102.0
14,deberta_m2_c2,98.0
38,deberta_m6_c2,81.0
36,deberta_m6_c0,73.0


Top 10 least important features

In [8]:
df.tail(10)

,name,importance
106,sentence_error_count_len_goe_11,0.0
109,word_common_percentage,0.0
110,word_rare_percentage,0.0
111,word_mistake_percentage,0.0
71,paragraph_error_count_len_goe_1,0.0
70,paragraph_error_count_len_l_1,0.0
62,paragraph_sentence_count_len_l_1,0.0
55,paragraph_error_count_q3,0.0
120,grammar_levenshtein_distance_min,0.0
144,grammar_cosine_distance_q3,0.0


## Broad Groups

In [9]:
def get_broad_group(name):
    group = name.split("_")[0]

    if group == "tfidf" and name.split("_")[1] == "count":
        return "tfidf_count"

    return group

In [10]:
df["broad_group"] = df["name"].map(get_broad_group)

In [11]:
px.box(
    df,
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

From an high level, it looks like
* `score` probabilities from DeBERTa model has he highest importance.
* `Paragraph` and `sentence` features also seem to be quite important as well.
* `word` and `tfidf_count` seems to be the least important.

In [12]:
def get_count_type(name):
    return "_".join(name.split("_")[1:3])

In [13]:
df["count_type"] = df["name"].map(get_count_type)

## Broad Operations

In [14]:
def get_operation_type(name):
    last_word = name.split("_")[-1]
    if last_word in [
        "sum",
        "max",
        "mean",
        "min",
        "q1",
        "q3",
    ]:
        return last_word

    if name.find("len_goe") != -1:
        return "greater_than"
    elif name.find("len_l") != -1:
        return "less_than"
    return "None"

In [15]:
df["operation_type"] = df["name"].map(get_operation_type)

In [16]:
px.box(
    df,
    x="importance",
    color="operation_type",
    range_x=(df.importance.min(), df.importance.max()),
)

Broadly, `min`, `kurtosis`, `q1`, `first` are worthless, regardless of broad group.

### Greater Than

In [17]:
px.box(
    df[df.operation_type == "greater_than"],
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

In [18]:
px.box(
    df[df.operation_type == "greater_than"],
    x="importance",
    color="count_type",
    range_x=(df.importance.min(), df.importance.max()),
)

In [19]:
df[(df.operation_type == "greater_than") & (df.importance >= 10)]

,name,importance,broad_group,count_type,operation_type
69,paragraph_sentence_count_len_goe_16,29.0,paragraph,sentence_count,greater_than
77,paragraph_error_count_len_goe_16,20.0,paragraph,error_count,greater_than


**Verdict**

1. For paragraphs
   1. Count only sentence and error. Upper limit be 15.
2. For sentences
   1. Count only error and word. Upper limit be 20.

### Less Than

In [20]:
px.box(
    df[df.operation_type == "less_than"],
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

In [21]:
px.box(
    df[df.operation_type == "greater_than"],
    x="importance",
    color="count_type",
    range_x=(df.importance.min(), df.importance.max()),
)

In [22]:
df[(df.operation_type == "less_than") & (df.importance >= 10)]

,name,importance,broad_group,count_type,operation_type
68,paragraph_sentence_count_len_l_16,41.0,paragraph,sentence_count,less_than
76,paragraph_error_count_len_l_16,24.0,paragraph,error_count,less_than


**Verdict**

1. For paragraphs: Count only sentence and error. Upper limit be 20.
2. For sentences: Count only error and word. Upper limit be 20.

## Paragraph Features

In [23]:
paragraph_features = df.loc[df.broad_group == "paragraph"]
print("Shape of paragraph features:", paragraph_features.shape)

Shape of paragraph features: (36, 5)


In [24]:
px.box(
    paragraph_features,
    x="importance",
    color="count_type",
    range_x=(df.importance.min(), df.importance.max()),
)

In [25]:
paragraph_features[
    (paragraph_features.count_type == "char_count")
    & (paragraph_features.importance > 0)
]

,name,importance,broad_group,count_type,operation_type
47,paragraph_char_count_sum,34.0,paragraph,char_count,sum
46,paragraph_char_count_max,32.0,paragraph,char_count,max
45,paragraph_char_count_mean,25.0,paragraph,char_count,mean
57,paragraph_char_count_q3,21.0,paragraph,char_count,q3
56,paragraph_char_count_q1,11.0,paragraph,char_count,q1


In [26]:
paragraph_features[
    (paragraph_features.count_type == "word_count")
    & (paragraph_features.importance > 0)
]

,name,importance,broad_group,count_type,operation_type
50,paragraph_word_count_sum,46.0,paragraph,word_count,sum
48,paragraph_word_count_mean,35.0,paragraph,word_count,mean
49,paragraph_word_count_max,29.0,paragraph,word_count,max
59,paragraph_word_count_q3,22.0,paragraph,word_count,q3
58,paragraph_word_count_q1,10.0,paragraph,word_count,q1


## DeBERTa Features

In [27]:
deberta_features = df.loc[df.broad_group.isin(["deberta"])]
print("Shape of deberta features:", deberta_features.shape)

Shape of deberta features: (42, 5)


In [28]:
def get_model_num(name):
    return int(name.split("_")[1][1])


deberta_features["model"] = deberta_features.name.map(get_model_num)

/tmp/ipykernel_11138/962180890.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
px.box(
    deberta_features,
    x="importance",
    color="model",
    range_x=(df.importance.min(), df.importance.max()),
)

## Word Features

In [30]:
word_features = df.loc[df.broad_group.isin(["word"])]
print("Shape of word features:", word_features.shape)

Shape of word features: (8, 5)


In [31]:
def get_specific(name):
    return ' '.join(name.split("_")[1:])

word_features["specific"] = word_features["name"].map(get_specific)

/tmp/ipykernel_11138/1087019223.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
word_features

,name,importance,broad_group,count_type,operation_type,specific
113,word_consonent_score,37.0,word,consonent_score,None,consonent score
112,word_scrabble_scores,35.0,word,scrabble_scores,None,scrabble scores
114,word_syllable_score,22.0,word,syllable_score,None,syllable score
115,word_unique_syllable_score,20.0,word,unique_syllable,None,unique syllable score
116,word_syllable_rarity_score,15.0,word,syllable_rarity,None,syllable rarity score
109,word_common_percentage,0.0,word,common_percentage,None,common percentage
110,word_rare_percentage,0.0,word,rare_percentage,None,rare percentage
111,word_mistake_percentage,0.0,word,mistake_percentage,None,mistake percentage


In [33]:
px.box(
    word_features,
    x="importance",
    color="specific",
    # range_x=(df.importance.min(), df.importance.max()),
)

## Grammar Features

In [34]:
grammar_features = df.loc[df.broad_group.isin(["grammar"])]
print("Shape of grammar features:", grammar_features.shape)

Shape of grammar features: (28, 5)


In [38]:
def get_specific(name):
    return name.split("_")[1]

grammar_features["specific"] = grammar_features["name"].map(get_specific)

/tmp/ipykernel_11138/2658045460.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
fig = px.box(
    grammar_features,
    x="importance",
    color="specific",
    range_x=(df.importance.min(), df.importance.max()),
)
fig.update_layout(
    height=1080,
    width=1920,
    title_x=0.5,
    title_text=f"Grammar Feature Importances<br><sup>Type Overview</sup>",
    legend=dict(orientation="h", yanchor="top", xanchor="center", y=-0.2, x=0.5),
    barmode="group",
)
fig.show()

In [46]:
fig = px.box(
    grammar_features,
    x="importance",
    color="operation_type",
    range_x=(df.importance.min(), df.importance.max()),
)
fig.update_layout(
    height=1080,
    width=1920,
    title_x=0.5,
    title_text=f"Grammar Feature Importances<br><sup>Operation Type Overview</sup>",
    legend=dict(orientation="h", yanchor="top", xanchor="center", y=-0.2, x=0.5),
    barmode="group",
)
fig.show()